In [1]:
from os import path, makedirs
from pickle import dump, load
from re import compile, sub
from requests import get, exceptions
from itertools import chain
from nltk.tokenize import sent_tokenize
from json import JSONDecodeError

Регулярки

In [8]:
def cut(data):
    r = compile(r'<.*?>|>>\d*|\(OP\)|&#(\d*);|&quot;|&gt;|(http|https):.*')
    return r.sub('', punctuate_word(punctuate_sent((data))))

def punctuate_sent(data):
    r = compile(r'([a-zA-Zа-яА-Я])([.!\?])')
    return r.sub(r'\1. ', data)

def punctuate_word(data):
    r = compile(r'([a-zA-Zа-яА-Я])([,])')
    return r.sub(r'\1, ', data)

Получение списка тредов доски

In [21]:
DVACH = 'https://2ch.hk/'
BOARD = 'b'

try:
    dvach_page = get(DVACH + BOARD + '/catalog.json').json()
    threads = [i['num'] for i in dvach_page['threads']]
except JSONDecodeError:
    print('No board with such name.')

Формирование корпуса сообщений доски

In [23]:
documents = []

for every_thread in threads:
    try:
        thread = get(DVACH +  BOARD + '/res/' + every_thread + '.json', timeout=5).json()
        [documents.append(sent_tokenize(cut(i['comment']))) for i in thread['threads'][0]['posts'] if len(cut(i['comment'])) > 2]
    except JSONDecodeError or ConnectionError:
        pass
    
documents = list(chain.from_iterable(documents))

Загрузка старого корпуса и мердж

In [13]:
try:
    file = open(path.join('pickle', BOARD + '.pickle'),'rb')
    old_documents = load(file)
    documents.extend(x for x in old_documents if x not in documents)
except FileNotFoundError:
    print('No pre-serialized data!')

Сериализация корпуса

In [24]:
subdir = 'pickle'

if not path.exists(subdir):
    makedirs(subdir)
file_path = path.join(subdir, BOARD + '.pickle')

with open(file_path, 'wb') as f:
    dump(documents, f)